# Feature Engineering en SQL

A continuación, veremos cómo calcular diferentes variables para el feature engineering utilizando SQL.


In [41]:
%pip install duckdb
%pip install jupysql
%pip install duckdb-engine

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///../../../db/dm_eyf.db

In [2]:
dataset_path = 'C:/Users/Federico/Desktop/Maestria Data mining/DM EyF/datasets/'
dataset_file = 'competencia_01.csv'

In [3]:
%%sql
create or replace table competencia_01 as
select
    *
from read_csv_auto("../../../datasets/competencia_01.csv")

,Success


In [4]:
df=pd.read_csv(dataset_path + dataset_file)

In [5]:
campos=df.columns.to_list()
print(campos[-1])
campos.pop(-1) #elimino clase ternaria
print(campos[-1])

clase_ternaria
Visa_mpagominimo


Ahora creo una tabla con las nuevas columnas, tengo que hacer la macro por que estoy dividiendo por 0.. 

In [6]:
%%sql
CREATE OR REPLACE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0)

,Success


In [7]:
%%sql
CREATE OR REPLACE MACRO div_sin_cero(a, b) as
    case
        when ifnull(b, 0) = 0 THEN NULL
        else ifnull(a, 0) / ifnull(b, 1)
    end

,Success


In [8]:
# Separar las columnas por tipo
master = [col for col in campos if 'Master' in col]
visa = [col for col in campos if 'Visa' in col]

sumas = []

for m_col, v_col in zip(master, visa):
    # Crear la suma usando la función 'suma_sin_null'
    suma = f"\n    suma_sin_null({m_col}, {v_col}) as {m_col.replace('Master_', '')}_total"
    sumas.append(suma)

consulta_sql = f"""
select 
    numero_de_cliente,
    {', '.join(sumas)}
from competencia_01;
"""

print(consulta_sql)


select 
    numero_de_cliente,
    
    suma_sin_null(Master_delinquency, Visa_delinquency) as delinquency_total, 
    suma_sin_null(Master_status, Visa_status) as status_total, 
    suma_sin_null(Master_mfinanciacion_limite, Visa_mfinanciacion_limite) as mfinanciacion_limite_total, 
    suma_sin_null(Master_Fvencimiento, Visa_Fvencimiento) as Fvencimiento_total, 
    suma_sin_null(Master_Finiciomora, Visa_Finiciomora) as Finiciomora_total, 
    suma_sin_null(Master_msaldototal, Visa_msaldototal) as msaldototal_total, 
    suma_sin_null(Master_msaldopesos, Visa_msaldopesos) as msaldopesos_total, 
    suma_sin_null(Master_msaldodolares, Visa_msaldodolares) as msaldodolares_total, 
    suma_sin_null(Master_mconsumospesos, Visa_mconsumospesos) as mconsumospesos_total, 
    suma_sin_null(Master_mconsumosdolares, Visa_mconsumosdolares) as mconsumosdolares_total, 
    suma_sin_null(Master_mlimitecompra, Visa_mlimitecompra) as mlimitecompra_total, 
    suma_sin_null(Master_madelantopesos, Vi

In [60]:
%%sql
create or replace table competencia_01_suma_division as
select
    numero_de_cliente,
    foto_mes,
    suma_sin_null(Master_delinquency, Visa_delinquency) as delinquency_total,
    suma_sin_null(Master_status, Visa_status) as status_total,
    suma_sin_null(Master_mfinanciacion_limite, Visa_mfinanciacion_limite) as mfinanciacion_limite_total,
    suma_sin_null(Master_msaldototal, Visa_msaldototal) as msaldototal_total,
    suma_sin_null(Master_msaldopesos, Visa_msaldopesos) as msaldopesos_total,
    suma_sin_null(Master_msaldodolares, Visa_msaldodolares) as msaldodolares_total,
    suma_sin_null(Master_mconsumospesos, Visa_mconsumospesos) as mconsumospesos_total,
    suma_sin_null(Master_mconsumosdolares, Visa_mconsumosdolares) as mconsumosdolares_total,
    suma_sin_null(Master_mlimitecompra, Visa_mlimitecompra) as mlimitecompra_total,
    suma_sin_null(Master_madelantopesos, Visa_madelantopesos) as madelantopesos_total,
    suma_sin_null(Master_madelantodolares, Visa_madelantodolares) as madelantodolares_total,
    suma_sin_null(Master_mpagado, Visa_mpagado) as mpagado_total,
    suma_sin_null(Master_mpagospesos, Visa_mpagospesos) as mpagospesos_total,
    suma_sin_null(Master_mpagosdolares, Visa_mpagosdolares) as mpagosdolares_total,
    suma_sin_null(Master_mconsumototal, Visa_mconsumototal) as mconsumototal_total,
    suma_sin_null(Master_cconsumos, Visa_cconsumos) as cconsumos_total,
    suma_sin_null(Master_cadelantosefectivo, Visa_cadelantosefectivo) as cadelantosefectivo_total,
    suma_sin_null(Master_mpagominimo, Visa_mpagominimo) as mpagominimo_total,

    suma_sin_null(ctarjeta_visa, ctarjeta_master) as cant_tarjetas,
    suma_sin_null(ctarjeta_visa_transacciones, ctarjeta_master_transacciones) as cant_transac_tarjetas,
    suma_sin_null(mtarjeta_visa_consumo, mtarjeta_master_consumo) as mtarjetas_consumo,
    suma_sin_null(cprestamos_personales,cprestamos_prendarios) as c_prestamos,
    suma_sin_null(mpayroll, mpayroll2) as m_payroll_total, -- monto acreditacion
    suma_sin_null(cpayroll_trx, cpayroll2_trx) as m_cpayroll_total, -- cantidad de acreditaciones
    suma_sin_null(minversion1_dolares, minversion1_pesos) as minversion1_total,
    suma_sin_null(mplazo_fijo_dolares, mplazo_fijo_pesos) as mplazofijo_total,
    suma_sin_null(suma_sin_null(cseguro_vida, cseguro_auto), suma_sin_null(cseguro_vivienda, cseguro_accidentes_personales)) as cseguros_total,

    div_sin_cero(mconsumototal_total, mlimitecompra_total) as ratio_uso_credito,
    div_sin_cero(mpagado_total, mconsumototal_total) as ratio_pago_vs_consumo,
    div_sin_cero(mpagominimo_total, mpagado_total) as ratio_pago_min_vs_total,
    div_sin_cero(mconsumospesos_total, mconsumosdolares_total) as ratio_consumo_pesos_dolares,
    div_sin_cero(madelantopesos_total + madelantodolares_total, mlimitecompra_total) as ratio_adelantos_vs_limite,
    div_sin_cero(cseguros_total, mconsumototal_total) as ratio_seguros_vs_consumo,

    greatest(Master_Finiciomora,Visa_Finiciomora) as fechamora_mayor,
    least(Master_Finiciomora,Visa_Finiciomora) as fechamora_menor,
    greatest(Master_fultimo_cierre,Visa_fultimo_cierre) as fechacierre_mayor,
    least(Master_fultimo_cierre,Visa_fultimo_cierre) as fechacierre_menor,
    greatest(Master_Fvencimiento, Visa_Fvencimiento) as fvencimiento_mayor,
    least(Master_Fvencimiento, Visa_Fvencimiento) as fvencimiento_menor,
    greatest(Master_fechaalta, Visa_fechaalta) as fechaalta_mayor,
    least(Master_fechaalta, Visa_fechaalta) as fechalta_menor,

    ntile(10) over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_cuantile
from competencia_01
where foto_mes=202104 or foto_mes=202106



,Success


In [9]:
%%sql
select 
    *
from competencia_01_suma_division


,numero_de_cliente,foto_mes,delinquency_total,status_total,mfinanciacion_limite_total,msaldototal_total,msaldopesos_total,msaldodolares_total,mconsumospesos_total,mconsumosdolares_total,...,ratio_seguros_vs_consumo,fechamora_mayor,fechamora_menor,fechacierre_mayor,fechacierre_menor,fvencimiento_mayor,fvencimiento_menor,fechaalta_mayor,fechalta_menor,cliente_antiguedad_6
0,576634596,202106,0,0,1298112.28,158235.84,175216.98,0.00,78367.77,1.80,...,0.000026,NaN,NaN,7.0,7.0,-183.0,-913.0,5313.0,5313.0,8
1,681808990,202106,0,0,88678.80,9119.07,9900.28,796.38,3173.90,8.19,...,0.000630,NaN,NaN,0.0,0.0,-1310.0,-2040.0,4174.0,4174.0,8
2,684878853,202106,0,0,6429567.70,189326.23,222079.66,0.00,163971.42,6.28,...,0.000000,NaN,NaN,0.0,0.0,-548.0,-1279.0,4945.0,4945.0,8
3,686497329,202106,0,0,0.00,0.00,0.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8
4,687007886,202106,0,0,253368.00,7762.61,8604.57,500.96,493.41,3.69,...,0.002027,NaN,NaN,0.0,0.0,-1338.0,-1522.0,1428.0,496.0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981941,783904040,202102,0,0,2621276.70,155583.86,182499.86,0.00,174774.65,0.00,...,0.000000,NaN,NaN,-3.0,-3.0,-639.0,-1401.0,4819.0,1564.0,7
981942,784422111,202102,0,0,0.00,0.00,0.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
981943,785117545,202102,0,0,63342.00,4153.82,4153.82,0.00,0.00,0.00,...,NaN,NaN,NaN,4.0,4.0,-364.0,-364.0,2562.0,2562.0,7
981944,785196597,202102,0,0,608083.20,16471.23,19320.75,0.00,6024.85,0.00,...,0.000000,NaN,NaN,-3.0,-3.0,-609.0,-1035.0,4512.0,4100.0,7


In [11]:
campos = ["ctrx_quarter","cpayroll_trx","mpayroll","mpayroll2","cpayroll2_trx","mpasivos_margen",'mprestamos_personales','mcuentas_saldo','mcaja_ahorro','mcomisiones_mantenimiento',
'cproductos','cdescubierto_preacordado',"mtarjeta_visa_consumo","cprestamos_personales", "mcuenta_corriente"]

## NTILE

In [53]:
nuevos_ratios = ""
for campo in campos:
    # Añadir las columnas de lag y delta que ya tienes
    nuevos_ratios += f"\n    , ntile(10) over (partition by foto_mes order by {campo}) as {campo}_cuantile"

consulta_sql_Ntile = f"""
CREATE OR REPLACE TABLE competencia_01_NTILE AS
select
    numero_de_cliente,
    foto_mes
    {nuevos_ratios.strip()}
from competencia_01;
"""

print(consulta_sql_Ntile)


CREATE OR REPLACE TABLE competencia_01_NTILE AS
select
    numero_de_cliente,
    foto_mes
    , ntile(10) over (partition by foto_mes order by ctrx_quarter) as ctrx_quarter_cuantile
    , ntile(10) over (partition by foto_mes order by cpayroll_trx) as cpayroll_trx_cuantile
    , ntile(10) over (partition by foto_mes order by mpayroll) as mpayroll_cuantile
    , ntile(10) over (partition by foto_mes order by mpayroll2) as mpayroll2_cuantile
    , ntile(10) over (partition by foto_mes order by cpayroll2_trx) as cpayroll2_trx_cuantile
    , ntile(10) over (partition by foto_mes order by mpasivos_margen) as mpasivos_margen_cuantile
    , ntile(10) over (partition by foto_mes order by mprestamos_personales) as mprestamos_personales_cuantile
    , ntile(10) over (partition by foto_mes order by mcuentas_saldo) as mcuentas_saldo_cuantile
    , ntile(10) over (partition by foto_mes order by mcaja_ahorro) as mcaja_ahorro_cuantile
    , ntile(10) over (partition by foto_mes order by mcomisiones

In [54]:
%%sql
CREATE OR REPLACE TABLE competencia_01_NTILE AS
select
    numero_de_cliente,
    foto_mes
    , ntile(10) over (partition by foto_mes order by ctrx_quarter) as ctrx_quarter_cuantile
    , ntile(10) over (partition by foto_mes order by cpayroll_trx) as cpayroll_trx_cuantile
    , ntile(10) over (partition by foto_mes order by mpayroll) as mpayroll_cuantile
    , ntile(10) over (partition by foto_mes order by mpayroll2) as mpayroll2_cuantile
    , ntile(10) over (partition by foto_mes order by cpayroll2_trx) as cpayroll2_trx_cuantile
    , ntile(10) over (partition by foto_mes order by mpasivos_margen) as mpasivos_margen_cuantile
    , ntile(10) over (partition by foto_mes order by mprestamos_personales) as mprestamos_personales_cuantile
    , ntile(10) over (partition by foto_mes order by mcuentas_saldo) as mcuentas_saldo_cuantile
    , ntile(10) over (partition by foto_mes order by mcaja_ahorro) as mcaja_ahorro_cuantile
    , ntile(10) over (partition by foto_mes order by mcomisiones_mantenimiento) as mcomisiones_mantenimiento_cuantile
    , ntile(10) over (partition by foto_mes order by cproductos) as cproductos_cuantile
    , ntile(10) over (partition by foto_mes order by cdescubierto_preacordado) as cdescubierto_preacordado_cuantile
    , ntile(10) over (partition by foto_mes order by mtarjeta_visa_consumo) as mtarjeta_visa_consumo_cuantile
    , ntile(10) over (partition by foto_mes order by cprestamos_personales) as cprestamos_personales_cuantile
    , ntile(10) over (partition by foto_mes order by mcuenta_corriente) as mcuenta_corriente_cuantile
from competencia_01
where foto_mes =202104 or foto_mes =202106



,Success


## Lag delta r_diff

In [55]:
nuevos_ratios = ""
for campo in campos:
    # Añadir las columnas de lag y delta que ya tienes
    nuevos_ratios += f"\n    , lag({campo}, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_{campo}"
    nuevos_ratios += f"\n    , {campo} - lag_1_{campo} as delta_1_{campo}"
    nuevos_ratios += f"\n    , lag({campo}, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_{campo}"
    nuevos_ratios += f"\n    , {campo} - lag_2_{campo} as delta_2_{campo}"

    # Agregar la lógica del ratio entre los deltas
    #nuevos_ratios += f"\n    , case when lag_2_{campo} <> 0 then delta_1_{campo} / delta_2_{campo} else null end as ratio_{campo}"
    nuevos_ratios += f"\n    , div_sin_cero(delta_1_{campo},delta_2_{campo}) as ratio_{campo}"
consulta_sql_ratios = f"""
CREATE OR REPLACE TABLE competencia_01_lag_delta_ratios AS
select
    numero_de_cliente,
    foto_mes
    {nuevos_ratios.strip()}
from competencia_01;
"""

print(consulta_sql_ratios)


CREATE OR REPLACE TABLE competencia_01_lag_delta_ratios AS
select
    numero_de_cliente,
    foto_mes
    , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
    , ctrx_quarter - lag_1_ctrx_quarter as delta_1_ctrx_quarter
    , lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
    , ctrx_quarter - lag_2_ctrx_quarter as delta_2_ctrx_quarter
    , div_sin_cero(delta_1_ctrx_quarter,delta_2_ctrx_quarter) as ratio_ctrx_quarter
    , lag(cpayroll_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpayroll_trx
    , cpayroll_trx - lag_1_cpayroll_trx as delta_1_cpayroll_trx
    , lag(cpayroll_trx, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cpayroll_trx
    , cpayroll_trx - lag_2_cpayroll_trx as delta_2_cpayroll_trx
    , div_sin_cero(delta_1_cpayroll_trx,delta_2_cpayroll_trx) as ratio_cpayroll_trx
    , lag(mpayroll, 1) over (partition by numero_de_c

In [37]:
%%sql
CREATE OR REPLACE TABLE competencia_01_lag_delta_ratios AS
select
    numero_de_cliente,
    foto_mes
    , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
    , ctrx_quarter - lag_1_ctrx_quarter as delta_1_ctrx_quarter
    , lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
    , ctrx_quarter - lag_2_ctrx_quarter as delta_2_ctrx_quarter
    , div_sin_cero(delta_1_ctrx_quarter,delta_2_ctrx_quarter) as ratio_ctrx_quarter
    , lag(cpayroll_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpayroll_trx
    , cpayroll_trx - lag_1_cpayroll_trx as delta_1_cpayroll_trx
    , lag(cpayroll_trx, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cpayroll_trx
    , cpayroll_trx - lag_2_cpayroll_trx as delta_2_cpayroll_trx
    , div_sin_cero(delta_1_cpayroll_trx,delta_2_cpayroll_trx) as ratio_cpayroll_trx
    , lag(mpayroll, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpayroll
    , mpayroll - lag_1_mpayroll as delta_1_mpayroll
    , lag(mpayroll, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mpayroll
    , mpayroll - lag_2_mpayroll as delta_2_mpayroll
    , div_sin_cero(delta_1_mpayroll,delta_2_mpayroll) as ratio_mpayroll
    , lag(mpayroll2, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpayroll2
    , mpayroll2 - lag_1_mpayroll2 as delta_1_mpayroll2
    , lag(mpayroll2, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mpayroll2
    , mpayroll2 - lag_2_mpayroll2 as delta_2_mpayroll2
    , div_sin_cero(delta_1_mpayroll2,delta_2_mpayroll2) as ratio_mpayroll2
    , lag(cpayroll2_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpayroll2_trx
    , cpayroll2_trx - lag_1_cpayroll2_trx as delta_1_cpayroll2_trx
    , lag(cpayroll2_trx, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cpayroll2_trx
    , cpayroll2_trx - lag_2_cpayroll2_trx as delta_2_cpayroll2_trx
    , div_sin_cero(delta_1_cpayroll2_trx,delta_2_cpayroll2_trx) as ratio_cpayroll2_trx
    , lag(mpasivos_margen, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpasivos_margen
    , mpasivos_margen - lag_1_mpasivos_margen as delta_1_mpasivos_margen
    , lag(mpasivos_margen, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mpasivos_margen
    , mpasivos_margen - lag_2_mpasivos_margen as delta_2_mpasivos_margen
    , div_sin_cero(delta_1_mpasivos_margen,delta_2_mpasivos_margen) as ratio_mpasivos_margen
    , lag(mprestamos_personales, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mprestamos_personales
    , mprestamos_personales - lag_1_mprestamos_personales as delta_1_mprestamos_personales
    , lag(mprestamos_personales, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mprestamos_personales
    , mprestamos_personales - lag_2_mprestamos_personales as delta_2_mprestamos_personales
    , div_sin_cero(delta_1_mprestamos_personales,delta_2_mprestamos_personales) as ratio_mprestamos_personales
    , lag(mcuentas_saldo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuentas_saldo
    , mcuentas_saldo - lag_1_mcuentas_saldo as delta_1_mcuentas_saldo
    , lag(mcuentas_saldo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcuentas_saldo
    , mcuentas_saldo - lag_2_mcuentas_saldo as delta_2_mcuentas_saldo
    , div_sin_cero(delta_1_mcuentas_saldo,delta_2_mcuentas_saldo) as ratio_mcuentas_saldo
    , lag(mcaja_ahorro, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcaja_ahorro
    , mcaja_ahorro - lag_1_mcaja_ahorro as delta_1_mcaja_ahorro
    , lag(mcaja_ahorro, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcaja_ahorro
    , mcaja_ahorro - lag_2_mcaja_ahorro as delta_2_mcaja_ahorro
    , div_sin_cero(delta_1_mcaja_ahorro,delta_2_mcaja_ahorro) as ratio_mcaja_ahorro
    , lag(mcomisiones_mantenimiento, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcomisiones_mantenimiento
    , mcomisiones_mantenimiento - lag_1_mcomisiones_mantenimiento as delta_1_mcomisiones_mantenimiento
    , lag(mcomisiones_mantenimiento, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcomisiones_mantenimiento
    , mcomisiones_mantenimiento - lag_2_mcomisiones_mantenimiento as delta_2_mcomisiones_mantenimiento
    , div_sin_cero(delta_1_mcomisiones_mantenimiento,delta_2_mcomisiones_mantenimiento) as ratio_mcomisiones_mantenimiento
    , lag(cproductos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cproductos
    , cproductos - lag_1_cproductos as delta_1_cproductos
    , lag(cproductos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cproductos
    , cproductos - lag_2_cproductos as delta_2_cproductos
    , div_sin_cero(delta_1_cproductos,delta_2_cproductos) as ratio_cproductos
    , lag(cdescubierto_preacordado, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cdescubierto_preacordado
    , cdescubierto_preacordado - lag_1_cdescubierto_preacordado as delta_1_cdescubierto_preacordado
    , lag(cdescubierto_preacordado, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cdescubierto_preacordado
    , cdescubierto_preacordado - lag_2_cdescubierto_preacordado as delta_2_cdescubierto_preacordado
    , div_sin_cero(delta_1_cdescubierto_preacordado,delta_2_cdescubierto_preacordado) as ratio_cdescubierto_preacordado
    , lag(mtarjeta_visa_consumo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_visa_consumo
    , mtarjeta_visa_consumo - lag_1_mtarjeta_visa_consumo as delta_1_mtarjeta_visa_consumo
    , lag(mtarjeta_visa_consumo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mtarjeta_visa_consumo
    , mtarjeta_visa_consumo - lag_2_mtarjeta_visa_consumo as delta_2_mtarjeta_visa_consumo
    , div_sin_cero(delta_1_mtarjeta_visa_consumo,delta_2_mtarjeta_visa_consumo) as ratio_mtarjeta_visa_consumo
    , lag(cprestamos_personales, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cprestamos_personales
    , cprestamos_personales - lag_1_cprestamos_personales as delta_1_cprestamos_personales
    , lag(cprestamos_personales, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cprestamos_personales
    , cprestamos_personales - lag_2_cprestamos_personales as delta_2_cprestamos_personales
    , div_sin_cero(delta_1_cprestamos_personales,delta_2_cprestamos_personales) as ratio_cprestamos_personales
    , lag(mcuenta_corriente, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuenta_corriente
    , mcuenta_corriente - lag_1_mcuenta_corriente as delta_1_mcuenta_corriente
    , lag(mcuenta_corriente, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcuenta_corriente
    , mcuenta_corriente - lag_2_mcuenta_corriente as delta_2_mcuenta_corriente
    , div_sin_cero(delta_1_mcuenta_corriente,delta_2_mcuenta_corriente) as ratio_mcuenta_corriente
from competencia_01
where foto_mes =202104 or foto_mes =202106



,Success


In [36]:
%%sql
select *
from competencia_01_lag_delta_ratios
where foto_mes =202104 or foto_mes =202106


,numero_de_cliente,foto_mes,lag_1_ctrx_quarter,delta_1_ctrx_quarter,lag_2_ctrx_quarter,delta_2_ctrx_quarter,ratio_ctrx_quarter,lag_1_cpayroll_trx,delta_1_cpayroll_trx,lag_2_cpayroll_trx,...,lag_1_cprestamos_personales,delta_1_cprestamos_personales,lag_2_cprestamos_personales,delta_2_cprestamos_personales,ratio_cprestamos_personales,lag_1_mcuenta_corriente,delta_1_mcuenta_corriente,lag_2_mcuenta_corriente,delta_2_mcuenta_corriente,ratio_mcuenta_corriente
0,249221109,202104,174.0,14.0,161.0,27.0,0.518519,2.0,-1.0,1.0,...,0.0,0.0,0.0,0.0,NaN,-485.88,485.88,0.00,0.00,NaN
1,249278864,202104,110.0,0.0,NaN,NaN,NaN,1.0,0.0,NaN,...,0.0,0.0,NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN
2,249296729,202104,63.0,8.0,55.0,16.0,0.500000,2.0,-1.0,0.0,...,0.0,0.0,0.0,0.0,NaN,2990.46,-2990.46,3196.71,-3196.71,0.935481
3,249470734,202104,116.0,-4.0,112.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,0.00,0.00,0.00,0.00,NaN
4,249590846,202104,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,0.00,0.00,0.00,0.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328961,1588492803,202104,9.0,39.0,NaN,NaN,NaN,0.0,0.0,NaN,...,0.0,0.0,NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN
328962,1588725801,202104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
328963,1588924016,202104,2.0,8.0,NaN,NaN,NaN,0.0,2.0,NaN,...,0.0,0.0,NaN,NaN,NaN,0.00,-1416.04,NaN,NaN,NaN
328964,1591753879,202104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Ahora voy con el promedio de cada campo, haciendo dev

In [38]:
nuevos_features = ""
for campo in campos:
    # Calcular el promedio móvil de los últimos 3 meses
    nuevos_features += f"\n    , avg({campo}) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_{campo}"
    
    # Calcular la desviación del valor actual con respecto al promedio móvil
    nuevos_features += f"\n    , {campo} - avg_3_{campo} as dev_3_{campo}"

# Construir la consulta SQL
consulta_sql = f"""
CREATE OR REPLACE TABLE competencia_01_avg AS
select
    numero_de_cliente
    , foto_mes 
    {nuevos_features.strip()}
FROM competencia_01
"""

print(consulta_sql)


CREATE OR REPLACE TABLE competencia_01_avg AS
select
    numero_de_cliente
    , foto_mes 
    , avg(ctrx_quarter) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ctrx_quarter
    , ctrx_quarter - avg_3_ctrx_quarter as dev_3_ctrx_quarter
    , avg(cpayroll_trx) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cpayroll_trx
    , cpayroll_trx - avg_3_cpayroll_trx as dev_3_cpayroll_trx
    , avg(mpayroll) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mpayroll
    , mpayroll - avg_3_mpayroll as dev_3_mpayroll
    , avg(mpayroll2) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mpayroll2
    , mpayroll2 - avg_3_mpayroll2 as dev_3_mpayroll2
    , avg(cpayroll2_trx) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) a

In [40]:
%%sql

CREATE OR REPLACE TABLE competencia_01_avg_dev AS
select
    numero_de_cliente
    , foto_mes 
    , avg(ctrx_quarter) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ctrx_quarter
    , ctrx_quarter - avg_3_ctrx_quarter as dev_3_ctrx_quarter
    , avg(cpayroll_trx) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cpayroll_trx
    , cpayroll_trx - avg_3_cpayroll_trx as dev_3_cpayroll_trx
    , avg(mpayroll) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mpayroll
    , mpayroll - avg_3_mpayroll as dev_3_mpayroll
    , avg(mpayroll2) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mpayroll2
    , mpayroll2 - avg_3_mpayroll2 as dev_3_mpayroll2
    , avg(cpayroll2_trx) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cpayroll2_trx
    , cpayroll2_trx - avg_3_cpayroll2_trx as dev_3_cpayroll2_trx
    , avg(mpasivos_margen) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mpasivos_margen
    , mpasivos_margen - avg_3_mpasivos_margen as dev_3_mpasivos_margen
    , avg(mprestamos_personales) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mprestamos_personales
    , mprestamos_personales - avg_3_mprestamos_personales as dev_3_mprestamos_personales
    , avg(mcuentas_saldo) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcuentas_saldo
    , mcuentas_saldo - avg_3_mcuentas_saldo as dev_3_mcuentas_saldo
    , avg(mcaja_ahorro) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcaja_ahorro
    , mcaja_ahorro - avg_3_mcaja_ahorro as dev_3_mcaja_ahorro
    , avg(mcomisiones_mantenimiento) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcomisiones_mantenimiento
    , mcomisiones_mantenimiento - avg_3_mcomisiones_mantenimiento as dev_3_mcomisiones_mantenimiento
    , avg(cproductos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cproductos
    , cproductos - avg_3_cproductos as dev_3_cproductos
    , avg(cdescubierto_preacordado) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cdescubierto_preacordado
    , cdescubierto_preacordado - avg_3_cdescubierto_preacordado as dev_3_cdescubierto_preacordado
    , avg(mtarjeta_visa_consumo) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mtarjeta_visa_consumo
    , mtarjeta_visa_consumo - avg_3_mtarjeta_visa_consumo as dev_3_mtarjeta_visa_consumo
    , avg(cprestamos_personales) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cprestamos_personales
    , cprestamos_personales - avg_3_cprestamos_personales as dev_3_cprestamos_personales
    , avg(mcuenta_corriente) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcuenta_corriente
    , mcuenta_corriente - avg_3_mcuenta_corriente as dev_3_mcuenta_corriente
FROM competencia_01
where foto_mes =202104 or foto_mes =202106


,Success


In [41]:
%%sql
select 
    *
from competencia_01_avg_dev
where foto_mes =202104 or foto_mes =202106

,numero_de_cliente,foto_mes,avg_3_ctrx_quarter,dev_3_ctrx_quarter,avg_3_cpayroll_trx,dev_3_cpayroll_trx,avg_3_mpayroll,dev_3_mpayroll,avg_3_mpayroll2,dev_3_mpayroll2,...,avg_3_cproductos,dev_3_cproductos,avg_3_cdescubierto_preacordado,dev_3_cdescubierto_preacordado,avg_3_mtarjeta_visa_consumo,dev_3_mtarjeta_visa_consumo,avg_3_cprestamos_personales,dev_3_cprestamos_personales,avg_3_mcuenta_corriente,dev_3_mcuenta_corriente
0,249239632,202104,101.0,0.0,2.0,0.0,131036.310,0.000,0.0,0.0,...,10.0,0.0,1.0,0.0,37011.050,0.000,1.0,0.0,-8791.200,0.000
1,249239632,202106,114.0,13.0,2.5,0.5,163807.745,32771.435,0.0,0.0,...,10.0,0.0,1.0,0.0,40504.395,3493.345,1.0,0.0,-32136.245,-23345.045
2,249305414,202104,383.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,...,8.0,0.0,1.0,0.0,121069.580,0.000,0.0,0.0,584019.100,0.000
3,249305414,202106,368.0,-15.0,0.0,0.0,0.000,0.000,0.0,0.0,...,8.0,0.0,1.0,0.0,118586.465,-2483.115,0.0,0.0,625473.300,41454.200
4,249325175,202104,124.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,...,7.0,0.0,1.0,0.0,79095.400,0.000,0.0,0.0,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328961,1345972754,202106,145.0,0.0,2.0,0.0,73005.170,0.000,0.0,0.0,...,7.0,0.0,1.0,0.0,27913.520,0.000,1.0,0.0,-3201.080,0.000
328962,1346128894,202104,77.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,...,7.0,0.0,1.0,0.0,0.000,0.000,0.0,0.0,-280.820,0.000
328963,1346128894,202106,93.0,16.0,0.0,0.0,0.000,0.000,0.0,0.0,...,7.0,0.0,1.0,0.0,0.000,0.000,0.0,0.0,-1234.225,-953.405
328964,1346568349,202104,113.0,0.0,1.0,0.0,0.000,0.000,0.0,0.0,...,7.0,0.0,1.0,0.0,13722.360,0.000,0.0,0.0,0.000,0.000


## reg_slope

In [17]:
nuevos_features = ""
for campo in campos:
  nuevos_features +=  f"\n    ,regr_slope({campo}, cliente_antiguedad) over ventana_3 as {campo}_slope_3"


consulta_sql = f"""
CREATE OR REPLACE TABLE competencia_01_regr_slope AS
select
    numero_de_cliente
    , foto_mes 
    {nuevos_features.strip()}
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)
"""

print(consulta_sql)


CREATE OR REPLACE TABLE competencia_01_regr_slope AS
select
    numero_de_cliente
    , foto_mes 
    ,regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_3 as ctrx_quarter_slope_3
    ,regr_slope(cpayroll_trx, cliente_antiguedad) over ventana_3 as cpayroll_trx_slope_3
    ,regr_slope(mpayroll, cliente_antiguedad) over ventana_3 as mpayroll_slope_3
    ,regr_slope(mpayroll2, cliente_antiguedad) over ventana_3 as mpayroll2_slope_3
    ,regr_slope(cpayroll2_trx, cliente_antiguedad) over ventana_3 as cpayroll2_trx_slope_3
    ,regr_slope(mpasivos_margen, cliente_antiguedad) over ventana_3 as mpasivos_margen_slope_3
    ,regr_slope(mprestamos_personales, cliente_antiguedad) over ventana_3 as mprestamos_personales_slope_3
    ,regr_slope(mcuentas_saldo, cliente_antiguedad) over ventana_3 as mcuentas_saldo_slope_3
    ,regr_slope(mcaja_ahorro, cliente_antiguedad) over ventana_3 as mcaja_ahorro_slope_3
    ,regr_slope(mcomisiones_mantenimiento, cliente_antiguedad) over ventana_3 as mco

In [59]:
%%sql
CREATE OR REPLACE TABLE competencia_01_regr_slope AS
SELECT
    numero_de_cliente,
    foto_mes,
    regr_slope(ctrx_quarter, cliente_antiguedad) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) AS ctrx_quarter_slope_3,
    regr_slope(cpayroll_trx, cliente_antiguedad) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) AS cpayroll_trx_slope_3,
    regr_slope(mpayroll, cliente_antiguedad) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) AS mpayroll_slope_3,
    regr_slope(mpayroll2, cliente_antiguedad) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) AS mpayroll2_slope_3,
    regr_slope(cpayroll2_trx, cliente_antiguedad) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) AS cpayroll2_trx_slope_3,
    regr_slope(mpasivos_margen, cliente_antiguedad) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) AS mpasivos_margen_slope_3,
    regr_slope(mprestamos_personales, cliente_antiguedad) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) AS mprestamos_personales_slope_3,
    regr_slope(mcuentas_saldo, cliente_antiguedad) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) AS mcuentas_saldo_slope_3,
    regr_slope(mcaja_ahorro, cliente_antiguedad) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) AS mcaja_ahorro_slope_3,
    regr_slope(mcomisiones_mantenimiento, cliente_antiguedad) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) AS mcomisiones_mantenimiento_slope_3,
    regr_slope(cproductos, cliente_antiguedad) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) AS cproductos_slope_3,
    regr_slope(cdescubierto_preacordado, cliente_antiguedad) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) AS cdescubierto_preacordado_slope_3,
    regr_slope(mtarjeta_visa_consumo, cliente_antiguedad) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) AS mtarjeta_visa_consumo_slope_3,
    regr_slope(cprestamos_personales, cliente_antiguedad) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) AS cprestamos_personales_slope_3,
    regr_slope(mcuenta_corriente, cliente_antiguedad) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) AS mcuenta_corriente_slope_3
    
FROM competencia_01
WHERE foto_mes = 202104 or foto_mes = 202106


,Success


In [45]:
%%sql
select 
    *
from competencia_01_regr_slope

,numero_de_cliente,foto_mes,ctrx_quarter_slope_3,cpayroll_trx_slope_3,mpayroll_slope_3,mpayroll2_slope_3,cpayroll2_trx_slope_3,mpasivos_margen_slope_3,mprestamos_personales_slope_3,mcuentas_saldo_slope_3,mcaja_ahorro_slope_3,mcomisiones_mantenimiento_slope_3,cproductos_slope_3,cdescubierto_preacordado_slope_3,mtarjeta_visa_consumo_slope_3,cprestamos_personales_slope_3,mcuenta_corriente_slope_3
0,249236712,202104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,249236712,202106,7.0,1.5,33776.380,0.0,0.0,2658.790,-6811.340,46007.245,215339.710,0.0,0.0,0.0,0.000,0.0,0.00
2,249286746,202104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,249286746,202106,-50.5,0.0,0.000,0.0,0.0,-440.505,0.000,-7544.835,-369.850,0.0,-1.0,0.0,-10401.670,0.0,-3352.53
4,249291844,202104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328961,1255350666,202104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
328962,1255350666,202106,16.5,0.5,26835.925,0.0,0.0,74.695,-11980.285,32.675,2634.510,0.0,0.0,0.0,0.000,0.0,0.00
328963,1255400546,202104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
328964,1255400546,202106,-5.0,-1.0,4747.130,0.0,0.0,0.695,-5817.940,-1759.000,-658.625,0.0,0.0,0.0,8460.725,0.0,-1301.09


In [11]:
# campos = ["ctrx_quarter","cpayroll_trx","mpayroll","mpayroll2","cpayroll2_trx","mpasivos_margen",'mprestamos_personales','mcuentas_saldo','mcaja_ahorro','mcomisiones_mantenimiento',
# 'cproductos','cdescubierto_preacordado',"mtarjeta_visa_consumo","cprestamos_personales", "mcuenta_corriente", "Master_mlimitecompra","Visa_mlimitecompra"]

In [12]:
nuevos_features = ""
for campo in campos:
  nuevos_features +=  f"\n    ,percent_rank() OVER (ORDER BY {campo}) as percent_rank_{campo}"


consulta_sql = f"""
CREATE OR REPLACE TABLE competencia_01_percent_rank AS
select
    numero_de_cliente
    , foto_mes 
    {nuevos_features.strip()}
from competencia_01

"""

print(consulta_sql)


CREATE OR REPLACE TABLE competencia_01_percent_rank AS
select
    numero_de_cliente
    , foto_mes 
    ,percent_rank() OVER (ORDER BY ctrx_quarter) as percent_rank_ctrx_quarter
    ,percent_rank() OVER (ORDER BY cpayroll_trx) as percent_rank_cpayroll_trx
    ,percent_rank() OVER (ORDER BY mpayroll) as percent_rank_mpayroll
    ,percent_rank() OVER (ORDER BY mpayroll2) as percent_rank_mpayroll2
    ,percent_rank() OVER (ORDER BY cpayroll2_trx) as percent_rank_cpayroll2_trx
    ,percent_rank() OVER (ORDER BY mpasivos_margen) as percent_rank_mpasivos_margen
    ,percent_rank() OVER (ORDER BY mprestamos_personales) as percent_rank_mprestamos_personales
    ,percent_rank() OVER (ORDER BY mcuentas_saldo) as percent_rank_mcuentas_saldo
    ,percent_rank() OVER (ORDER BY mcaja_ahorro) as percent_rank_mcaja_ahorro
    ,percent_rank() OVER (ORDER BY mcomisiones_mantenimiento) as percent_rank_mcomisiones_mantenimiento
    ,percent_rank() OVER (ORDER BY cproductos) as percent_rank_cproductos
   

In [56]:
%%sql
CREATE OR REPLACE TABLE competencia_01_percent_rank AS
select
    numero_de_cliente
    , foto_mes 
    ,percent_rank() OVER (ORDER BY ctrx_quarter) as percent_rank_ctrx_quarter
    ,percent_rank() OVER (ORDER BY cpayroll_trx) as percent_rank_cpayroll_trx
    ,percent_rank() OVER (ORDER BY mpayroll) as percent_rank_mpayroll
    ,percent_rank() OVER (ORDER BY mpayroll2) as percent_rank_mpayroll2
    ,percent_rank() OVER (ORDER BY cpayroll2_trx) as percent_rank_cpayroll2_trx
    ,percent_rank() OVER (ORDER BY mpasivos_margen) as percent_rank_mpasivos_margen
    ,percent_rank() OVER (ORDER BY mprestamos_personales) as percent_rank_mprestamos_personales
    ,percent_rank() OVER (ORDER BY mcuentas_saldo) as percent_rank_mcuentas_saldo
    ,percent_rank() OVER (ORDER BY mcaja_ahorro) as percent_rank_mcaja_ahorro
    ,percent_rank() OVER (ORDER BY mcomisiones_mantenimiento) as percent_rank_mcomisiones_mantenimiento
    ,percent_rank() OVER (ORDER BY cproductos) as percent_rank_cproductos
    ,percent_rank() OVER (ORDER BY cdescubierto_preacordado) as percent_rank_cdescubierto_preacordado
    ,percent_rank() OVER (ORDER BY mtarjeta_visa_consumo) as percent_rank_mtarjeta_visa_consumo
    ,percent_rank() OVER (ORDER BY cprestamos_personales) as percent_rank_cprestamos_personales
    ,percent_rank() OVER (ORDER BY mcuenta_corriente) as percent_rank_mcuenta_corriente
    ,percent_rank() OVER (ORDER BY Master_mlimitecompra) as percent_rank_Master_mlimitecompra
    ,percent_rank() OVER (ORDER BY Visa_mlimitecompra) as percent_rank_Visa_mlimitecompra
from competencia_01
WHERE foto_mes = 202104 OR foto_mes = 202106



,Success


In [47]:
%%sql
select 
    *
from competencia_01_percent_rank

,numero_de_cliente,foto_mes,percent_rank_ctrx_quarter,percent_rank_cpayroll_trx,percent_rank_mpayroll,percent_rank_mpayroll2,percent_rank_cpayroll2_trx,percent_rank_mpasivos_margen,percent_rank_mprestamos_personales,percent_rank_mcuentas_saldo,percent_rank_mcaja_ahorro,percent_rank_mcomisiones_mantenimiento,percent_rank_cproductos,percent_rank_cdescubierto_preacordado,percent_rank_mtarjeta_visa_consumo,percent_rank_cprestamos_personales,percent_rank_mcuenta_corriente,percent_rank_Master_mlimitecompra,percent_rank_Visa_mlimitecompra
0,497190551,202104,0.264554,0.000000,0.000000,0.0,0.0,0.538130,0.000000,0.478385,0.703327,0.868770,0.237582,0.00000,0.150624,0.000000,0.436861,0.897971,0.000000
1,999816711,202106,0.152554,0.000000,0.000000,0.0,0.0,0.217111,0.000000,0.525445,0.310972,0.856909,0.237582,0.03542,0.151290,0.000000,0.436861,0.897971,0.000000
2,818781988,202104,0.041102,0.000000,0.000000,0.0,0.0,0.123542,0.000000,0.294475,0.228222,0.018011,0.044126,0.00000,0.170280,0.000000,0.436861,0.897971,0.000000
3,829423214,202104,0.407052,0.000000,0.000000,0.0,0.0,0.163662,0.962662,0.316082,0.321752,0.018011,0.013615,0.00000,0.000000,0.930816,0.436861,0.897971,0.000000
4,829423214,202106,0.355673,0.000000,0.000000,0.0,0.0,0.450252,0.961887,0.466062,0.646348,0.018011,0.013615,0.00000,0.000000,0.930816,0.436861,0.897971,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328961,319673097,202104,0.583506,0.453346,0.695834,0.0,0.0,0.873491,0.000000,0.856681,0.925144,0.018011,0.044126,0.03542,0.000000,0.000000,0.436861,0.480373,0.949967
328962,319673097,202106,0.463849,0.453346,0.741018,0.0,0.0,0.908373,0.000000,0.892566,0.945897,0.018011,0.044126,0.03542,0.000000,0.000000,0.436861,0.480373,0.949967
328963,637263533,202104,0.913596,0.700628,0.711073,0.0,0.0,0.923463,0.000000,0.927941,0.954372,0.018011,0.044126,0.03542,0.000000,0.000000,0.436861,0.480373,0.949967
328964,818164805,202106,0.209116,0.453346,0.578910,0.0,0.0,0.962455,0.000000,0.887642,0.962303,0.018011,0.237582,0.03542,0.000000,0.000000,0.436861,0.480373,0.949967


In [ ]:
## cree 5 tablas
## las sumas que me parecian mas unos ratios..
## otro con lag y delta de dos meses
## promedio de todos los campos
## regr_slope de todos los campos
## per_Rank

##.. ahora tengo que unir esto

In [63]:
%%sql 
create or replace table competencia_01_fe_2 as
SELECT * 
FROM (
    SELECT * 
    FROM competencia_01 
    WHERE foto_mes = 202104 OR foto_mes = 202106
) AS base

LEFT JOIN competencia_01_NTILE as ntile
ON base.numero_de_cliente = ntile.numero_de_cliente 
    AND base.foto_mes = ntile.foto_mes  

LEFT JOIN competencia_01_suma_division as suma
ON ntile.numero_de_cliente = suma.numero_de_cliente 
    AND ntile.foto_mes = suma.foto_mes  

LEFT JOIN competencia_01_lag_delta_ratios as lag  
ON suma.numero_de_cliente = lag.numero_de_cliente 
    AND suma.foto_mes = lag.foto_mes  

LEFT JOIN competencia_01_avg_dev as avg 
ON suma.numero_de_cliente = avg.numero_de_cliente 
    AND suma.foto_mes = avg.foto_mes  
    
LEFT JOIN competencia_01_regr_slope as reg 
ON suma.numero_de_cliente = reg.numero_de_cliente 
    AND suma.foto_mes = reg.foto_mes

LEFT JOIN competencia_01_percent_rank as p_rank
ON suma.numero_de_cliente = p_rank.numero_de_cliente 
    AND suma.foto_mes = p_rank.foto_mes

,Success


In [2]:
%%sql
COPY competencia_01_fe_2 TO '../../../datasets/competencia_01_fe_2.csv' (FORMAT CSV, HEADER TRUE)

,Success


In [4]:
%%sql
CREATE OR REPLACE TABLE competencia_01_fe AS
SELECT numero_de_cliente, foto_mes
FROM competencia_01 
LEFT JOIN competencia_01_suma_division 
USING (numero_de_cliente, foto_mes)
LEFT JOIN competencia_01_lag_delta 
USING (numero_de_cliente, foto_mes)
LEFT JOIN competencia_01_avg 
USING (numero_de_cliente, foto_mes)
LEFT JOIN competencia_01_regr_slope 
USING (numero_de_cliente, foto_mes)
LEFT JOIN competencia_01_percent_rank 
USING (numero_de_cliente, foto_mes)

,Success


In [23]:
# %%sql
# CREATE OR REPLACE TABLE competencia_01_fe AS
# SELECT *
# FROM competencia_01_suma_division as suma
# LEFT JOIN competencia_01_lag_delta as lag
# USING (numero_de_cliente, foto_mes)
# LEFT JOIN competencia_01_avg as avg
# USING (numero_de_cliente, foto_mes)
# LEFT JOIN competencia_01_regr_slope as reg
# USING (numero_de_cliente, foto_mes)
# LEFT JOIN competencia_01_percent_rank as p_rank
# USING (numero_de_cliente, foto_mes)

,Success


In [8]:
%%sql
COPY competencia_01_fe TO '../../../datasets/competencia_01_fe.csv' (FORMAT CSV, HEADER TRUE)

,Success


In [17]:
%%sql
select 
    foto_mes,foto_mes_1
from competencia_01_fe
limit 2

,foto_mes,foto_mes_1
0,202105,202105
1,202101,202101
